In [4]:
def get_nodes(co_list,col):
    '''
        rows: 所有用来构建合作网络的数据行
        col: 作者信息所在列数-1
    '''
    nodes_list = []
    for authors in co_list:
        auths = authors[col].split(" | ") # 分隔符可换
        for auth in auths:
            if auth not in nodes_list: # 去重
                nodes_list.append(auth)
    return nodes_list

In [46]:
def create_network(rows,name_list,col):
    '''
        rows: 形如co_list的所有用来构建合作网络的数据行
        name_list: 出现的作者（节点）列表
        col: 作者信息所在列数-1
    '''

    length = len(name_list)
    cooperate_list = []
    links = []
    # 遍历作者名,计数和各作者合作数(每次合作对自己也计数)
    for name in name_list:
        author_co = []
        author_co.append(name)
    # 合作列表：初始化为长度为节点个数的全零列表[name,0,0,0,...,0,0]
        for i in range(length):
            author_co.append(0) 
    # 遍历所有数据行,对作者字段进行匹配
        for row in rows:
            names_row = row[col].split(" | ") # 分隔符按数据结构选
    # 对包含当前作者的数据项,提取合作关系(合作列表对应位置+1)
            if name in names_row:
                for the_name in names_row:
                # 索引合作的作者编号,确定在合作列表中的位置
                    name_num = name_list.index(the_name)+1 
                # 对应位置+1
                    author_co[name_num] = author_co[name_num] + 1 
                # 获取边列表，作为提取“边-权重”列表和邻接表的输入
                    if name!=the_name:
                        link = sorted([name,the_name])
                        links.append(link)
    # 添加到合作矩阵列表中
        cooperate_list.append(author_co)

    # 1. 生成合作矩阵：csv_list
    csv_list = []
    first_line = []
    first_line.append('')
    first_line.extend(name_list)
    csv_list.append(first_line) # 第一行,第一列为标题(出现的作者名)
    csv_list.extend(cooperate_list)

    # 2. 生成”边-权重“表csv_links
    csv_links = get_weighted_links(links)
    
    # 3. 邻接表adj_links
    adj_links  = get_adjacency(links)
    #print(collections.Counter(adj_links))
    b = tuple(tuple([y for y in x]) for x in adj_links)
    print(collections.Counter(b))
#     test=pd.DataFrame(data=csv_links)
#     print(test)
#     test.to_csv('testcsv.csv',encoding='gbk')

In [13]:
def get_weighted_links(edges_list):
    ''' edges_list：是包含边的列表,[[节点1,节点2],...],边可以重复,重复次数记为权重
        返回值：带权重的边列表,[[节点1,节点2,weight],...]
    '''
    weighted_links = [] # 作为返回值
    exist_links = [] # 记录当前已经出现过的边
    for link in edges_list:
        if link not in exist_links:
            exist_links.append(link)
            link.append(1) # 新边weight置1
            weighted_links.append(link)            
        else:
            # exsit_links和weighted_links里的列表顺序是一样的
            index = exist_links.index(link) 
            # 已存在的边,weight + 1
            weighted_links[index][2] = weighted_links[index][2] + 1 
    return weighted_links
def get_adjacency(links):
    ''' links：是包含边列表的矩阵,[[作者1,作者2],...],边可以重复,重复次数记为权重
        返回值：邻接矩阵,[[作者1,作者2],...]
    '''
    adj_list = []
    weighted_links = get_weighted_links(links)
    for link in weighted_links:
        num = int(link[2])
        for i in range(num):
            adj_list.append(link[:2])
    
    return adj_list

In [47]:
import pandas as pd
if __name__ =='__main__' :

    co_list = [ ["AA | BB | CC | DD",2019],
                ["EE | BB | FF ",2018],
                ["AA | GG | FF | HH | KK",2019],
                ["CC | DD | FF | LL | AA",2020],
                ["AA | BB | FF ",2017],
                ["EE | BB | GG ",2018],
                ["DD | GG | LL | HH | EE",2019],
                ["AA | GG | CC | DD",2018]]

    # 获取节点列表
    author_list = get_nodes(co_list,0) 
    # 合作网络（生成：合作矩阵，边-权重表，邻接表）
    create_network(co_list,author_list,0)

Counter({('AA', 'CC'): 6, ('AA', 'DD'): 6, ('CC', 'DD'): 6, ('AA', 'BB'): 4, ('AA', 'GG'): 4, ('AA', 'FF'): 4, ('BB', 'EE'): 4, ('BB', 'FF '): 4, ('DD', 'LL'): 4, ('DD', 'GG'): 4, ('GG', 'HH'): 4, ('AA', 'HH'): 2, ('AA', 'KK'): 2, ('AA', 'LL'): 2, ('AA', 'FF '): 2, ('BB', 'CC'): 2, ('BB', 'DD'): 2, ('BB', 'GG '): 2, ('CC', 'FF'): 2, ('CC', 'LL'): 2, ('CC', 'GG'): 2, ('DD', 'FF'): 2, ('DD', 'HH'): 2, ('DD', 'EE'): 2, ('EE', 'FF '): 2, ('EE', 'GG '): 2, ('EE', 'GG'): 2, ('EE', 'LL'): 2, ('EE', 'HH'): 2, ('FF', 'GG'): 2, ('GG', 'KK'): 2, ('GG', 'LL'): 2, ('FF', 'HH'): 2, ('FF', 'KK'): 2, ('FF', 'LL'): 2, ('HH', 'KK'): 2, ('HH', 'LL'): 2})


In [35]:
import itertools
import collections
l = "apple banana banana apple".split()
def pairwise(iterable):
    """s -> (s0,s1), (s1,s2), (s2, s3), ..."""
    a, b = itertools.tee(iterable)
    next(b, None)
    return ((a, b) if a < b else (b, a) for a, b in zip(a, b))
list(pairwise(l))

[('apple', 'banana'), ('banana', 'banana'), ('apple', 'banana')]

In [36]:
collections.Counter(pairwise(l))

Counter({('apple', 'banana'): 2, ('banana', 'banana'): 1})

In [48]:
# -*-  coding: utf-8 -*-
import os, sys
import jieba, codecs, math
import jieba.posseg as pseg
 
names = {}            # 姓名字典
relationships = {}    # 关系字典
lineNames = []        # 每段内人物关系

In [51]:
jieba.load_userdict("E:\ROST+系列人文社科研究大数据计算工具\ROSTCM6\dict.txt")        # 加载字典
with codecs.open("E:\ROST+系列人文社科研究大数据计算工具\ROSTCM6\\busan.txt", "r", "utf8") as f:
    for line in f.readlines():
        poss = pseg.cut(line)        # 分词并返回该词词性
        lineNames.append([])        # 为新读入的一段添加人物名称列表
        for w in poss:
            if w.flag != "nr" or len(w.word) < 2:
                continue            # 当分词长度小于2或该词词性不为nr时认为该词不为人名
            lineNames[-1].append(w.word)        # 为当前段的环境增加一个人物
            if names.get(w.word) is None:
                names[w.word] = 0
                relationships[w.word] = {}
            names[w.word] += 1                    # 该人物出现次数加 1

In [56]:
# calc relations
relations = {}
f = (x for x in codecs.open("E:\ROST+系列人文社科研究大数据计算工具\ROSTCM6\\busan.txt", 'r', 'utf-8'))
for line in f:
    line = line.strip()
    if not line: continue
    segment = line.split('。')
    for content in segment:
        # put into set
        relation = set()
        for name in names:
            if name in content: relation.add(name)
        if len(relation) <= 1: continue
        # process set to list
        temp = []
        for x in relation:
            temp.append(x)
        relation = temp
        # process relations
        for index, i in enumerate(relation):
            for j in relation[index + 1:]:
                key = str(i) + ' ' + str(j)
                if not relations.get(key):
                    relations[key] = 1
                else:
                    relations[key] += 1
relations = sorted(relations.items(), key=lambda x: x[1], reverse=True)

In [57]:
print(relations)

[('秀安 石宇', 53), ('尚华 石宇', 37), ('尚华 盛京', 19), ('石宇 盛京', 19), ('英国 石宇', 17), ('珍熙 英国', 15), ('秀安 盛京', 15), ('石宇的妈妈 石宇', 12), ('秀安 露宿者', 10), ('英国 尚华', 10), ('金代理 石宇', 6), ('秀安 尚华', 6), ('海英 少女', 6), ('海英 组长', 6), ('露宿者 石宇', 6), ('队员 珍熙', 5), ('露宿者 盛京', 5), ('金常务 列车长', 4), ('金常务 乘务长', 4), ('金常务 石宇', 4), ('石宇的妈妈 秀安', 3), ('海英 乘务长', 3), ('队员 英国', 3), ('金代理 组长', 3), ('秀安 金常务', 3), ('珍熙 乘务长', 3), ('卡住 尚华', 3), ('珍熙 金常务', 3), ('英国 金常务', 3), ('队员 少女', 2), ('少女 珍熙', 2), ('英国 珍熙', 2), ('组长 石宇', 2), ('卡住 石宇', 2), ('露宿者 尚华', 2), ('珍熙 石宇', 2), ('英国 秀安', 2), ('金常务 盛京', 2), ('金代理 安全性', 1), ('蓝天 乌云', 1), ('秀安 麻麻亮', 1), ('麻麻亮 石宇', 1), ('队员 美少女', 1), ('美少女 少女', 1), ('美少女 珍熙', 1), ('珍熙 谢谢', 1), ('海英 呼唤', 1), ('呼唤 少女', 1), ('海英 小姐', 1), ('组长 小姐', 1), ('英国 少女', 1), ('海英 石宇', 1), ('塞进 尚华', 1), ('盛京 安静', 1), ('盛京 谢谢', 1), ('明白 尚华', 1), ('明白 石宇', 1), ('游弋 露宿者', 1), ('列车长 乘务长', 1), ('爱抚 尚华', 1), ('爱抚 盛京', 1), ('冷笑 金常务', 1), ('珍熙 麻木', 1), ('石宇 尚华', 1), ('珍熙 呼唤', 1), ('英国 呼唤', 1), ('英国 露宿者', 1), ('金常务 钟吉', 1), (

In [55]:
for name,times in names.items():
    print (name, times)

石宇 213
金代理 15
安全性 1
秀安 135
石宇的妈妈 12
和娜英 2
乌云 1
蓝天 1
黎明 15
麻麻亮 1
乘务长 15
海英 25
谢谢 3
列车长 20
安静 1
美少女 1
队员 10
珍熙 48
英国 60
老婆婆 1
钟吉 11
组长 17
少女 14
金常务 52
露宿者 22
尚华 112
呼唤 2
小姐 2
秀玉 1
盛京 58
塞进 1
卡住 4
游弋 4
明白 2
爱抚 1
冷笑 1
麻木 1
钟吉冲 1
拉进来 1
幸存者 1
苏醒过来 1
安抚 1
阿姨 1
士兵 2
